In [1]:
import prody
import openenm as oenm
import numpy as np

## OpenENM ANM

In [2]:
anm_oenm = oenm.AnisotropicNetworkModel('1tcd', cutoff='10 angstroms')

In [3]:
anm_oenm.hessian_matrix

array([[ 3.76049413, -0.28232553,  0.21635409, ...,  0.        ,
         0.        ,  0.        ],
       [-0.28232553,  4.03251558, -1.09164678, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.21635409, -1.09164678,  3.2069903 , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  2.57922086,
        -0.28738021,  0.31902976],
       [ 0.        ,  0.        ,  0.        , ..., -0.28738021,
         5.0640629 ,  1.07030712],
       [ 0.        ,  0.        ,  0.        , ...,  0.31902976,
         1.07030712,  5.35671624]])

## Prody GNM

In [4]:
tri = prody.parsePDB('1tcd')

@> PDB file is found in working directory (1tcd.pdb.gz).
@> 3983 atoms and 1 coordinate set(s) were parsed in 0.06s.


In [5]:
calphas = tri.select('calpha')

In [6]:
anm_prody = prody.ANM('Triosa')

### Kirchhoff matrix comparison

In [7]:
anm_prody.buildHessian(calphas, cutoff=10., gamma=1.)

@> Hessian was built in 0.12s.


In [8]:
prody_hessian = anm_prody.getHessian()

In [9]:
prody_hessian

array([[ 3.76049413, -0.28232553,  0.21635409, ...,  0.        ,
         0.        ,  0.        ],
       [-0.28232553,  4.03251558, -1.09164678, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.21635409, -1.09164678,  3.2069903 , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  2.57922086,
        -0.28738021,  0.31902976],
       [ 0.        ,  0.        ,  0.        , ..., -0.28738021,
         5.0640629 ,  1.07030712],
       [ 0.        ,  0.        ,  0.        , ...,  0.31902976,
         1.07030712,  5.35671624]])

In [10]:
np.allclose(anm_oenm.hessian_matrix, prody_hessian)

True

### Eigvals and Eigvects comparison

In [11]:
anm_prody.calcModes('all', zeros=True)

@> 1491 modes were calculated in 0.77s.


In [13]:
np.allclose(anm_oenm.eigenvalues, anm_prody.getEigvals())

True

In [15]:
eigvecs_prody = anm_prody.getEigvecs()

In [27]:
eigvecs_prody[:,6]

array([ 0.00801775, -0.03147705, -0.01586492, ..., -0.02975954,
        0.02208985,  0.03086396])

In [29]:
anm_oenm.eigenvectors[:,6]

array([ 0.00801775, -0.03147705, -0.01586492, ..., -0.02975954,
        0.02208985,  0.03086396])

In [21]:
np.diagonal(anm_oenm.eigenvectors)

array([ 2.63969877e-20,  7.52268133e-03, -6.18994951e-02, ...,
       -5.43040560e-04,  2.19047635e-04, -5.73107278e-04])

In [35]:
comparison=[]
for ii in range(5, 497):
    if np.sign(anm_oenm.eigenvectors[0][ii])!=np.sign(eigvecs_prody[0][ii]):
        comparison.append(np.allclose(anm_oenm.eigenvectors[:,ii], -eigvecs_prody[:,ii]))
    else:
        comparison.append(np.allclose(anm_oenm.eigenvectors[:,ii], eigvecs_prody[:,ii]))

In [44]:
prody.writeNMD('p38_anm_modes.nmd', anm_prody[0:10], calphas)

'p38_anm_modes.nmd'

In [42]:
prody.pathVMD()

@> WARNING VMD path is not set by user, looking for it.
@> VMD path is set to '/usr/local/bin/vmd'.


'/usr/local/bin/vmd'

In [43]:
prody.viewNMDinVMD('p38_anm_modes.nmd')


Info) VMD for LINUXAMD64, version 1.9.2 (December 29, 2014)
Info) http://www.ks.uiuc.edu/Research/vmd/                         
Info) Email questions and bug reports to vmd@ks.uiuc.edu           
Info) Please include this reference in published work using VMD:   
Info)    Humphrey, W., Dalke, A. and Schulten, K., `VMD - Visual   
Info)    Molecular Dynamics', J. Molec. Graphics 1996, 14.1, 33-38.
Info) -------------------------------------------------------------
Info) Multithreading available, 20 CPUs detected.
Info) Free system memory: 17399MB (54%)
Info) Creating CUDA device pool and initializing hardware...
Info) Detected 2 available CUDA accelerators:
Info) [0] NVIDIA GeForce GTX 1080 20 SM_6.1 @ 1.81 GHz, 7.9GB RAM, KTO, AE2, ZCP
Info) [1] Quadro M2000        6 SM_5.2 @ 1.16 GHz, 3.9GB RAM, KTO, AE1, ZCP
Warning) Detected X11 'Composite' extension: if incorrect display occurs
Warning) try disabling this X server option.  Most OpenGL drivers
Warning) disable stereoscopic display w